# Estimando $\beta$ de los activos

## <font color=green> Conclusiones de la lectura. </font>

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/4/48/Capital_market_line_of_CAPM.svg" width="400px" height="400px" />

En la clase anterior aprendimos 
- ¿qué es el CAPM?; 
- ¿cuáles son los supuestos sobre los que se funda el CAPM?;
- derivamos la fórmula del CAPM; y
- aprendimos como obtener la $\beta$ de un portafolio a partir de la $\beta$ de activos individuales.

En la clase de hoy estudiaremos una forma de estimar los $\beta$ de activos individuales.

**Objetivos:**
- Revisitar riesgo sistemático y no sistemático.
- Estudiar un método para estimar las $\beta$ de los activos.

*Referencia:*
- Notas del curso "Portfolio Selection and Risk Management", Rice University, disponible en Coursera.
- [Notas del curso "Financial Engineering", Columbia University](http://www.columbia.edu/~ks20/FE-Notes/FE-Notes-Sigman.html)
___

## 1. Riesgo sistemático y no sistemático.

Recordamos la fórmula de CAPM:

$$E[r_i]-r_f=\beta_i(E[r_M]-r_f),$$

donde $\beta_i=\frac{\sigma_{M,i}}{\sigma_M^2}$ y $\sigma_{M,i}$ es la covarianza del portafolio de mercado con el activo individual $i$.

Todas las anteriores son variables determinísiticas.

- ¿Qué pasa si usamos el CAPM como un modelo de rendimientos? Es decir,

$$r_i=r_f+\beta_i(r_M-r_f)+\epsilon_i,$$

donde $\epsilon_i$ es un término de error.

<font color=green> Despejando $\epsilon_i$, tenemos que: </font>

- $E[\epsilon_i]=0$, y
- $cov(\epsilon_i,r_M)=0$.

<font color=green> Ver en el tablero. </font>

Entonces, la varianza del activo $i$ es:

$$\sigma_i=\beta_i^2\sigma_M^2+var(\epsilon_i),$$

donde el primer término corresponde al riesgo sistemático (de mercado) y el segundo al riesgo idiosincrático.
___

## 2. Estimando $\beta$ para un activo.

- En el mercado real, la cantidad de activos es ENORME, y tratar de construir el portafolio de mercado sería una tarea grandiosa, pero poco realista para un analista financiero. 
- Por lo tanto, los índices de mercado han sido creados para intentar aproximar el portafolio de mercado.
- Dicho índice es un portafolio más pequeño que el de mercado, construido por los que se consideran los activos más dominantes, y que capturan la esencia de el portafolio de mercado.

- El índice de mercado más conocido es el Standard & Poor’s 500-stock index (S&P), compuesto de 500 activos.
- Un $\beta$ para un activo dado, se puede estimar utilizando el S&P en reemplazo de M, y utilizando datos históricos para ambos rendimientos (el del activo y el del S&P500).
- Por ejemplo, considere un activo $i$ para el cual queremos estimar su $\beta_i$.
- Este estimado se construye usando medias, varianzas y covarianzas muestrales como sigue:
 - Escogemos $N$ rendimientos históricos, tales como los reportados mensualmente hace tres años.
 - Para $k=1,2,\dots,N$, $r_{ik}$ y $r_{S\&Pk}$ denotan el $k-$ésimo valor muestral de los rendimietos.

Entonces

$$\hat{E[r_i]}=\frac{1}{N}\sum_{k=1}^{N}r_{ik}, \text{ y}$$

$$\hat{E[r_{S\&P}]}=\frac{1}{N}\sum_{k=1}^{N}r_{S\&Pk}.$$

Además, la varianza $\sigma_{S\&P}^2$ se estima como

$$\hat{\sigma_{S\&P}^2}=\frac{1}{N-1}\sum_{k=1}^{N}(r_{S\&Pk}-\hat{E[r_{S\&P}]})^2,$$

y la covarianza $\sigma_{S\&P,i}$

$$\hat{\sigma_{S\&P,i}}=\frac{1}{N-1}\sum_{k=1}^{N}(r_{S\&Pk}-\hat{E[r_{S\&P}]})(r_{ik}-\hat{E[r_i]})$$

Finalmente, 

$$\hat{\beta_i}=\frac{\hat{\sigma_{S\&P,i}}}{\hat{\sigma_{S\&P}^2}}.$$

### Ejemplo...

Entrar a yahoo finance, a la información de MSFT, AAPL, GCARSOA1.MX y ^GSPC.

In [1]:
# Importar paquetes
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web

In [2]:
# Función para descargar precios de cierre ajustados de varios activos a la vez:
def get_closes(tickers, start_date=None, end_date=None, freq=None):
    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Frecuencia de muestreo por defecto (freq='d')
    # Importamos paquetes necesarios
    import pandas as pd
    pd.core.common.is_list_like = pd.api.types.is_list_like
    import pandas_datareader.data as web  
    # Creamos DataFrame vacío de precios, con el índice de las fechas
    closes = pd.DataFrame(columns = tickers, index=web.YahooDailyReader(symbols=tickers[0], start=start_date, end=end_date, interval=freq).read().index)
    # Agregamos cada uno de los precios con YahooDailyReader
    for ticker in tickers:
        df = web.YahooDailyReader(symbols=ticker, start=start_date, end=end_date, interval=freq).read()
        closes[ticker]=df['Adj Close']
    closes.index_name = 'Date'
    closes = closes.sort_index()
    return closes

In [3]:
# Importar datos de AAPL, MSFT, GCARSOA1.MX, y ^GSPC
names = ['AAPL', 'MSFT', 'GCARSOA1.MX', '^GSPC']
start_date = '2015-11-01'
end_date = '2018-11-07'

In [4]:
# Precios mensuales
monthly_closes = get_closes(names, start_date=start_date, end_date=end_date, freq='m')
monthly_closes

,AAPL,MSFT,GCARSOA1.MX,^GSPC
Date,,,,
2015-11-01,112.120659,50.740330,70.638725,2080.409912
2015-12-01,100.188835,52.144402,68.300453,2043.939941
2016-01-01,92.650391,51.777847,70.157600,1940.239990
2016-02-01,92.031723,47.820961,70.475136,1932.229980
2016-03-01,104.302032,52.282135,77.528427,2059.739990
2016-04-01,89.707985,47.208221,79.164253,2065.300049
2016-05-01,95.564751,50.171154,75.151665,2096.949951
2016-06-01,92.045006,48.777630,75.522438,2098.860107
2016-07-01,100.334824,54.030018,74.099861,2173.600098


In [5]:
# Rendimientos mensuales
monthly_ret = monthly_closes.pct_change().dropna()
monthly_ret

,AAPL,MSFT,GCARSOA1.MX,^GSPC
Date,,,,
2015-12-01,-0.106419,0.027672,-0.033102,-0.017530
2016-01-01,-0.075242,-0.007030,0.027191,-0.050735
2016-02-01,-0.006677,-0.076420,0.004526,-0.004128
2016-03-01,0.133327,0.093289,0.100082,0.065991
2016-04-01,-0.139921,-0.097049,0.021100,0.002699
2016-05-01,0.065287,0.062763,-0.050687,0.015325
2016-06-01,-0.036831,-0.027775,0.004934,0.000911
2016-07-01,0.090063,0.107680,-0.018836,0.035610
2016-08-01,0.018137,0.013761,0.000653,-0.001219


In [6]:
# Matriz de covarianza
cov = monthly_ret.cov()
cov

,AAPL,MSFT,GCARSOA1.MX,^GSPC
AAPL,0.005178,0.001881,-0.000747,0.000807
MSFT,0.001881,0.002246,-0.000087,0.000772
GCARSOA1.MX,-0.000747,-0.000087,0.004092,0.000235
^GSPC,0.000807,0.000772,0.000235,0.000718


In [8]:
# Beta de Microsoft
beta_MSFT = cov['MSFT']['^GSPC']/cov['^GSPC']['^GSPC']
np.round(beta_MSFT,2)

1.07

In [9]:
# Beta de Apple
beta_AAPL = cov['AAPL']['^GSPC']/cov['^GSPC']['^GSPC']
np.round(beta_AAPL,2)

1.12

In [10]:
# Beta de Grupo Carso
beta_GCARSO = cov['GCARSOA1.MX']['^GSPC']/cov['^GSPC']['^GSPC']
np.round(beta_GCARSO,2)

0.33

**Conclusiones**

[Índice para México](https://espanol.spindices.com/indices/equity/sp-bmv-ipc)
___

## 3. Ejercicios

Su portafolio de inversión consisten de $\$150,000$ invertidos en un solo activo. Suponga que la tasa de interés libre de riesgo es $3\%$. Este activo tiene un rendimiento esperado del $12\%$, y una volatilidad del $40\%$, y el portafolio de mercado tiene un rendimiento esperado del $10\%$ y una volatilidad del $18\%$. Bajo las suposiciones del CAPM, ¿cuál es la volatilidad de un mejor portafolio alternativo que tiene el mismo rendimiento que el activo?

A. $\sigma = 15.52\%$

B. $\sigma = 23.14\%$

C. $\sigma = 30.25\%$ 

D. $\sigma = 35.13\%$ 

Con los mismos datos del ejercicio anterior, y suponiendo que usted está conforme con el riesgo del activo, bajo las suposiciones de CAPM, ¿cuál sería un mejor rendimiento esperado que debería ganar?

A. $E[r] = 13.65\%$

B. $E[r] = 15.30\%$

C. $E[r] = 18.56\%$

D. $E[r] = 20.38\%$ 

Suponga que usted tiene invertidos $\$30,000$ en los siguientes cuatro activos:

| Activo        | Cantidad invertida | $\beta$ |
| ------------- | ------------------ | ------- |
| Activo A      | $\$5,000$          | 0.75    |
| Activo B      | $\$10,000$         | 1.10    |
| Activo C      | $\$8,000$          | 1.36    |
| Activo D      | $\$7,000$          | 1.88    |

La tasa libre de riesgo es del $2\%$ y el rendimiento esperado del portafolio de mercado es del $8\%$. Con base en el CAPM, ¿cuál es la $\beta$ del portafolio?

A. $\beta_P = 0.95$

B. $\beta_P = 1.19$

C. $\beta_P = 1.29$

D. $\beta_P = 1.62$

Considere la siguiente distribución de rendimientos:

| Estado de la economía | Probabilidad       | $R_A$   |
| --------------------- | ------------------ | ------- |
| Depresión             | 0.30               | 0.10    |
| Normal                | 0.50               | 0.15    |
| Expansión             | 0.20               | 0.25    |

Suponiendo que el CAPM es válido, que la volatilidad del portafolio de mercado es $\sigma_M=0.10$ y que la correlación del activo A y el portafolio de mercado es $\rho_{A,M}=0.9$, ¿cuál es el valor de $\beta_A$?

A. $\beta_A=0.25$

B. $\beta_A=0.47$

C. $\beta_A=0.55$

D. $\beta_A=1.15$

Un fondo de inversiones con $\beta=0.8$ tiene un rendimiento esperado del $16\%$. Si la tasa libre de riesgo es $4\%$ y el rendimiento esperado del portafolio de mercado es del $13\%$, ¿debería invertir en este fondo?

A. Si.

B. No.

Suponga que encontramos un activo el cual, con base en su precio al día de hoy, tiene un rendimiento esperado más bajo que lo que sugiere la línea de mercado de activos. Si el CAPM es válido, ¿cuál(es) de las siguientes proposiciones es (son) verdaderas?

A. Si el activo tiene un rendimiento esperado más bajo que el sugerido por la línea, significa que su precio es muy bajo.

B. Si el activo tiene un rendimiento esperado más bajo que el sugerido por la línea, significa que su precio es muy alto.

C. El activo se grafica bajo la línea.

D. El activo se grafica por encima de la línea.

# Anuncios parroquiales

## 1. Último quiz la próxima clase.
## 2. Próxima clase: asesoría de proyectos.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>